In [1]:
import pandas as pd
from geopy.distance import distance

In [2]:
raw_data = pd.read_csv('../raw_data/202106-citibike-tripdata.csv')

/tmp/ipykernel_82193/2419724043.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('../raw_data/202106-citibike-tripdata.csv')


In [3]:
df = raw_data.drop(['rideable_type', 'start_station_name', 'end_station_name', 'member_casual'], axis=1)
df['start_station_id'] = pd.to_numeric(df['start_station_id'], errors='coerce')
df['end_station_id'] = pd.to_numeric(df['end_station_id'], errors='coerce')
df['ended_at'] = pd.to_datetime(df['ended_at'])
df['started_at'] = pd.to_datetime(df['started_at'])

df.dtypes

ride_id                     object
started_at          datetime64[ns]
ended_at            datetime64[ns]
start_station_id           float64
end_station_id             float64
start_lat                  float64
start_lng                  float64
end_lat                    float64
end_lng                    float64
dtype: object

In [4]:
in_df = df.drop(['started_at', 'start_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng'], axis=1)
in_df = in_df[(in_df['ended_at'] >= '2021-06-01 00:00:00') & (in_df['ended_at'] <= '2021-06-30 23:59:59')]
in_df['station_id'] = df['end_station_id']
in_df['latitude'] = df['end_lat']
in_df['longitude'] = df['end_lng']
in_df = in_df.drop(['ended_at', 'end_station_id', 'ended_at'], axis=1)

out_df = df.drop(['ended_at', 'end_station_id','start_lat', 'start_lng', 'end_lat', 'end_lng'], axis=1)
out_df = out_df[(out_df['started_at'] >= '2021-06-01 00:00:00') & (out_df['started_at'] <= '2021-06-30 23:59:59')]
out_df['station_id'] = df['start_station_id']
out_df['latitude'] = df['start_lat']
out_df['longitude'] = df['start_lng']
out_df = out_df.drop(['started_at', 'start_station_id', 'started_at'], axis=1)

station_info = pd.concat([out_df, in_df])
station_info = station_info.groupby('station_id').mean().reset_index()

In [5]:
selected_stations = station_info['station_id'].unique()
distances  = pd.crosstab(station_info.station_id, station_info.station_id, normalize=True)
distances[:] = 0

for pos,station_i in enumerate(selected_stations[:-1]):
    if pos%368 ==0:
        print(str(pos/368*25)+'% complete')
    for station_j in selected_stations[pos+1:]:
        station_i_meta = station_info[station_info['station_id']==station_i]
        station_j_meta = station_info[station_info['station_id']==station_j]
        d_ij = distance(
            (station_i_meta['latitude'].values[0], station_i_meta['longitude'].values[0]), 
            (station_j_meta['latitude'].values[0], station_j_meta['longitude'].values[0])).m
        distances.at[station_j, station_i] = d_ij
        distances.at[station_i, station_j] = d_ij

0.0% complete
25.0% complete
50.0% complete
75.0% complete
100.0% complete


In [6]:
outbound_stations = pd.read_csv('../processed_data/outbound_stations.csv')
outbound_distances = distances[list(outbound_stations.station_id)]
outbound_distances = outbound_distances[outbound_distances.index.isin(outbound_stations.station_id)]
outbound_distances = outbound_distances.reindex(sorted(outbound_distances.columns), axis=1)
outbound_distances = outbound_distances.reindex(sorted(outbound_distances.index), axis=0)
outbound_distances.to_csv('../processed_data/outbound_distances.csv')
print('Outbound distances length:', len(outbound_distances))
print(print(pd.Series(outbound_distances.values.ravel()).describe()))

inbound_stations = pd.read_csv('../processed_data/inbound_stations.csv')
inbound_distances = distances[list(inbound_stations.station_id)]
inbound_distances = inbound_distances[inbound_distances.index.isin(inbound_stations.station_id)]
inbound_distances = inbound_distances.reindex(sorted(inbound_distances.columns), axis=1)
inbound_distances = inbound_distances.reindex(sorted(inbound_distances.index), axis=0)
inbound_distances.to_csv('../processed_data/inbound_distances.csv')
print('Inbound distances length:', len(inbound_distances))
print(print(pd.Series(inbound_distances.values.ravel()).describe()))

stations = pd.read_csv('../processed_data/stations.csv')
distances = distances[list(stations.station_id)]
distances = distances[distances.index.isin(stations.station_id)]
distances = distances.reindex(sorted(distances.columns), axis=1)
distances = distances.reindex(sorted(distances.index), axis=0)
distances.to_csv('../processed_data/distances.csv')
print('All distances length:', len(distances))
print(print(pd.Series(distances.values.ravel()).describe()))

Outbound distances length: 1473
count    2.169729e+06
mean     9.062130e+03
std      5.496552e+03
min      0.000000e+00
25%      4.742322e+03
50%      8.032432e+03
75%      1.272553e+04
max      2.972343e+04
dtype: float64
None
Inbound distances length: 1475
count    2.175625e+06
mean     9.061921e+03
std      5.496697e+03
min      0.000000e+00
25%      4.741743e+03
50%      8.032375e+03
75%      1.272620e+04
max      2.972343e+04
dtype: float64
None
All distances length: 1475
count    2.175625e+06
mean     9.061921e+03
std      5.496697e+03
min      0.000000e+00
25%      4.741743e+03
50%      8.032375e+03
75%      1.272620e+04
max      2.972343e+04
dtype: float64
None
